In [1]:
from Bio.Entrez import efetch, read

def fetch_abstract(pmid):
    handle = efetch(db='pubmed', id=pmid, retmode='xml')
    xml_data = read(handle)['PubmedArticle'][0]
    try:
        article = xml_data['MedlineCitation']['Article']
        abstract = article['Abstract']['AbstractText'][0]
        return abstract
    except IndexError:
        return None

def fetch_title(pmid):
    handle = efetch(db='pubmed', id=pmid, retmode='xml')
    xml_data = read(handle)['PubmedArticle'][0]
    try:
        article_title = xml_data['MedlineCitation']['Article']['ArticleTitle']
        return article_title
    except IndexError:
        return None

In [29]:
!pip install Biopython

In [2]:
import pandas as pd
articles = pd.read_csv('extra/data/PubMed/oa_noncomm_xml.PMC001xxxxxx.baseline.2022-12-18.filelist.txt', delimiter='\t')

In [3]:
import psycopg2
conn = psycopg2.connect("dbname=postgres user=postgres password=gui0721")

for id in articles['PMID']:
     try:
        cur = conn.cursor()
        id = str(id)
        title = fetch_title(id)
        print(title)
        sql = """INSERT INTO pubmed(id, title)
             VALUES(%s, %s);"""
        cur.execute(sql, (id, title))
        
     except Exception as e:
         print(e)
         continue
conn.commit()


c:\ProgramData\Anaconda3\envs\research\lib\site-packages\Bio\Entrez\__init__.py:694: UserWarning: 
            Email address is not specified.

            To make use of NCBI's E-utilities, NCBI requires you to specify your
            email address with each request.  As an example, if your email address
            is A.N.Other@example.com, you can specify it as follows:
               from Bio import Entrez
               Entrez.email = 'A.N.Other@example.com'
            In case of excessive usage of the E-utilities, NCBI will attempt to contact
            a user at the email address provided before blocking access to the
            E-utilities.
  warnings.warn(


HTTP Error 500: Internal Server Error
Crucial role for ecto-5'-nucleotidase (CD73) in vascular leakage during hypoxia.
duplicate key value violates unique constraint "pubmed_pkey"
DETAIL:  Key (id)=(15583013) already exists.

HTTP Error 500: Internal Server Error
HTTP Error 500: Internal Server Error
HTTP Error 500: Internal Server Error
HTTP Error 500: Internal Server Error
HTTP Error 500: Internal Server Error
Sgk3 links growth factor signaling to maintenance of progenitor cells in the hair follicle.
current transaction is aborted, commands ignored until end of transaction block



KeyboardInterrupt: 

In [17]:
cur.execute("""CREATE TABLE pubmed (
                id INTEGER PRIMARY KEY,
                title VARCHAR(255) NOT NULL
        )""")
cur.close()
conn.commit()

In [27]:
cur = conn.cursor()
sql = """INSERT INTO pubmed(id, title)
             VALUES(%s, %s);"""
cur.execute(sql, (124, "test"))
conn.commit()

In [2]:
!pip install pymed

In [22]:
cur = conn.cursor()
id = str(id)
title = fetch_title(id)
print(title)
sql = """INSERT INTO pubmed(id, title)
        VALUES(%s, %s);"""
cur.execute(sql, (id, title))
conn.commit()

NameError: name 'fetch_title' is not defined

In [35]:


from pymed import PubMed
path = "retrieved_articles/"
# Create a PubMed object that GraphQL can use to query
# Note that the parameters are not required but kindly requested by PubMed Central
# https://www.ncbi.nlm.nih.gov/pmc/tools/developers/
pubmed = PubMed(tool="MyTool", email="my@email.address")

# Create a GraphQL query in plain text
query = 'sleep stress'

# Execute the query against the API
results = pubmed.query(query, max_results=3000)

# Loop over the retrieved articles
for article in results:
    print(article)

    # Extract and format information from the article
    article_id = article.pubmed_id
    title = article.title

    abstract = article.abstract
    print(article_id)
    print(article.title)
    print(article.abstract)

    # f = open(path + article_id + ".txt", "w")
    # f.write(abstract)
    # f.close()
    # id = str(article_id)
    # print(article_id)
    # cur = conn.cursor()
    # sql = """INSERT INTO pubmed(id, title) VALUES(%s, %s) ON CONFLICT (id) DO NOTHING;"""
    # cur.execute(sql, (id, title))
    # conn.commit()



36916105
Combined application of dupilumab and mite allergen-specific immunotherapy in children with moderate to severe atopic dermatitis.
Atopic dermatitis (AD) is a chronic, relapsing inflammatory skin disease that manifests in skin dryness, severe itching, and eczema, and can significantly impact a patient's quality of life. Current treatment regimens do not prevent the recurrence of the disease and are associated with adverse effects. Here, we report two cases of moderate-to-severe AD in children that were treated with dupilumab, a dual inhibitor of IL-4 and IL-13 signaling, in combination with mite allergen-specific immunotherapy.
Both patients presented with the diagnosis of AD that was not adequately controlled by the conventional treatment regimen, including topical corticosteroids (TCS), topical calcineurin inhibitors, emollients, and the traditional Chinese medicine treatments. In both patients, AD-associated skin irritation impacted the quality of life, disturbed sleep patte

KeyboardInterrupt: 

In [137]:
import psycopg2
conn = psycopg2.connect("dbname=postgres user=postgres password=gui0721")
cur = conn.cursor()

In [138]:
main_header = "PMID- "
abstract_header = "AB  - "
title_header = "TI  - "
abstract_in_progress = False
running_abstract = ""
title_in_progress = False
running_title = ""
cur = {}
path = "retrieved_articles/"

In [140]:
with open('pubmed-sleepstres-set.txt') as f:
    for line in f:
        if line.startswith(main_header):
            id = line[6:-1]
            cur = {'id': id}
        if line.startswith(title_header):
            title_in_progress = True
            running_title = line[6:-1]
        if title_in_progress and line.startswith("      "):
            running_title += line[6:-1]
        if title_in_progress and (not line.startswith("      ")) and (not line.startswith(title_header)):
            title_in_progress = False
            cur['title'] = running_title
        if line.startswith(abstract_header):
            abstract_in_progress = True
            running_abstract = line[6:-1]
        if abstract_in_progress and line.startswith("      "):
            running_abstract += line[6:-1]
        if abstract_in_progress and (not line.startswith("      ")) and (not line.startswith(abstract_header)):
            abstract_in_progress = False
            cur['abstract'] = running_abstract
            
            #save
            wf = open(path + cur['id'] + ".txt", "w")
            wf.write(cur['abstract'])
            wf.close()
            curs = conn.cursor()
            sql = """INSERT INTO pubmed(id, title) VALUES(%s, %s) ON CONFLICT (id) DO UPDATE SET title=%s;"""
            if not 'title' in cur:
                print("No title")
                title = ''
            else:
                title = cur['title']
            if len(title) >= 255:
                curs.execute(sql, (cur['id'], title[:255], title[:255]))
            else:
                curs.execute(sql, (cur['id'], title, title))

                
conn.commit()
            





No title
No title
No title
No title


In [20]:
import psycopg2
conn = psycopg2.connect("dbname=postgres user=postgres password=gui0721")
cur = conn.cursor()
import pandas as pd
f = pd.read_csv('titles2.csv', header=None)


In [21]:
commands = """
CREATE TABLE links (
    id TEXT PRIMARY KEY,
    link TEXT
)
"""

In [22]:
curs.execute(commands)
conn.commit()

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


In [10]:
for (idx, item) in enumerate(f[0]):
    print(idx + 1, item)
    title = item
    id = str(idx + 1)
    curs = conn.cursor()
    sql = """INSERT INTO pubmed(id, title) VALUES(%s, %s) ON CONFLICT (id) DO UPDATE SET title=%s;"""
    curs.execute(sql, (id, title, title))
conn.commit()

1 The validity, reliability, and utility of the iButton for measurement of body temperature circadian rhythms in sleep/wake research
2 The Temperature Dependence of Sleep
3 Distinct ipRGC subpopulations mediate light’s acute and circadian effects on body temperature and sleep
4 Circadian clock neurons constantly monitor environmental temperature to set sleep timing
5 The endogenous circadian temperature period length (tau) in delayed sleep phase disorder compared to good sleepers
6 Circadian Melatonin and Temperature Taus in Delayed Sleep-wake Phase Disorder and Non-24-hour Sleep-wake Rhythm Disorder Patients: An Ultradian Constant Routine Study
7 Circadian Impairment of Distal Skin Temperature Rhythm in Patients With Sleep-Disordered Breathing: The Effect of CPAP
8 Effects of Caffeine on Skin and Core Temperatures, Alertness, and Recovery Sleep During Circadian Misalignment
9 Evening physical activity alters wrist temperature circadian rhythmicity
10 Effects of the Melatonin MT-1/MT-2

Process data downloaded from spike

In [30]:
import pandas as pd
import psycopg2
import os
import numpy as np
conn = psycopg2.connect("dbname=postgres user=postgres password=gui0721")
data_file = pd.read_csv("results (1).csv")
path = "generated_files/"
for index, row in data_file.iterrows():
    curs = conn.cursor()
    link = row['article_link']
    title = row['title']
    text = row['paragraph_text']
    id = row['pmid']
    print(id)
    if np.isnan(id):
        print('here')
        id = row['pmcid'][3:]
    print(id)
    file_path = path + str(id) + '.txt'
    if os.path.exists(file_path):
        with open(file_path, 'a') as f:
            f.write(text)
            f.close()
    else:
        with open(file_path, 'x') as f:
            f.write(text)
            f.close()
    sql = """INSERT INTO pubmed(id, title) VALUES(%s, %s) ON CONFLICT (id) DO UPDATE SET title=%s;"""
    curs.execute(sql, (id, title, title))
        

    sql = """INSERT INTO links(id, link) VALUES(%s, %s) ON CONFLICT (id) DO UPDATE SET link=%s;"""
    curs.execute(sql, (id, link, link))

conn.commit()
    



33436540.0
33436540.0
11595619.0
11595619.0
29558391.0
29558391.0
22315516.0
22315516.0
31848393.0
31848393.0
32411680.0
32411680.0
34474513.0
34474513.0
31160381.0
31160381.0
33424933.0
33424933.0
16596305.0
16596305.0
31031659.0
31031659.0
29626158.0
29626158.0
33339284.0
33339284.0
26095528.0
26095528.0
19578504.0
19578504.0
19578504.0
19578504.0
nan
nan


NumericValueOutOfRange: integer out of range
